Sentence similarity
Dataset: STS

In [29]:
import torch
from datasets import load_dataset, Dataset
import nltk
import re
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses

from nltk.corpus import words
from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)
nltk.download('words', quiet=True)
nltk.download('wordnet', quiet=True)
# vocab = set(words.words())
from nltk.stem import *

from preprocessing import clean_sentence, vocab, clean_dataset

In [2]:
ds = load_dataset("tabilab/biosses")

In [3]:
train_test = ds['train'].train_test_split(test_size=0.2, seed=42)
train_validate = train_test['train'].train_test_split(test_size=0.2, seed=42)
train_set = train_validate['train']
validate_set = train_validate['test']
test_set = train_test['test']

In [4]:
stemmer = PorterStemmer()

In [5]:
stemmed_words = set(map(lambda word: stemmer.stem(word), words.words()))

In [6]:
cleaned_train, unknown_words = clean_dataset(train_set, stemmed_words)
cleaned_validation, _ = clean_dataset(validate_set, stemmed_words)

In [7]:
print(unknown_words)
print(len(unknown_words))

{'IDH1', 'let-7)', 'and/or', '3', 'tumor-suppressor', 'STK33', 'caspase', '(together', 'TargetScan', 'miRVec-miR-204', 'plasmid', '[8]', 'rate-limiting', 'mdm2', 'vivo', 'B-ALL', 'Lats2/Kpm', '286', 'miR-204-miRVec', 'oncogenesis]', '95%', 'anaphase-promoting', 'co-transfected', '(CD44', 'RNAhybrid', 'miR-145', 'tumorigenic', 'pSuper', 'TET1/2/3', '(PTC)', 'large-scale', 'miR-373', 'hmC', 'tumour', '10]', 'Affymetrix', 'RNA', 'StemBase', 'GTP-bound', 'miR-133b', 'signal-regulated', 'miR-16−1', 'miR-204', 'Up-regulation', 'WT1', 'shRNA-based', 'BAF', 'POU5F1', 'Thr288', 'SOX-17', 'ductal', 'LATS2', '(C/EBPα)', 'ribonucleotide', 'ng', 'loss-of-function', 'downregulated', 'RIPK1', 'OSCC', 'Oct-4', 'granulopoiesis', '7,000', 'K-Ras', 'genome-scale', 'I/A', 'HeLa', 'myeloid-specific', 'cycle-dependent', 'electroporation', 'Aurora-A', '(RIP1)', 'G-proteins', 'deregulation', 'Caco-2', 'D1', 'miR-146a', 'dysregulation', 'factor)', 'miR-34', 'RIP1', 'Toji', 'miR-15a', 'hepatocellular', 'ubiquit

In [8]:
max(cleaned_validation['score'])

1.0

In [9]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.losses import CoSENTLoss

# Load a model to train/finetune
model = SentenceTransformer("xlm-roberta-base")

# Initialize the CoSENTLoss
# This loss requires pairs of text and a float similarity score as a label
loss = CoSENTLoss(model)

No sentence-transformers model found with name xlm-roberta-base. Creating a new one with mean pooling.


In [18]:
train_s1 = cleaned_train['sentence1']
train_s2 = cleaned_train['sentence2']
validation_s1 = cleaned_validation['sentence1']
validation_s2 = cleaned_validation['sentence2']

In [17]:
train_emb1 = model.encode(train_s1)
train_emb2 = model.encode(train_s2)

In [19]:
validation_emb1 = model.encode(validation_s1)
validation_emb2 = model.encode(validation_s2)

In [24]:
validation_similarities = model.similarity_pairwise(validation_emb1, validation_emb2)

In [25]:
validation_similarities

tensor([0.9988, 0.9987, 0.9981, 0.9977, 0.9968, 0.9985, 0.9984, 0.9983, 0.9969,
        0.9959, 0.9972, 0.9993, 0.9974, 0.9983, 0.9976, 0.9969])

In [30]:
trainer = SentenceTransformerTrainer(
    model=model,
    train_dataset=cleaned_train,
    loss=loss,
)
trainer.train()

 25%|██▌       | 6/24 [00:40<01:47,  5.99s/it]

KeyboardInterrupt: 